In [ ]:
import urllib
import requests
from bs4 import BeautifulSoup
import os
import random
import urllib
from tqdm.notebook import tqdm
from collections import defaultdict
from timeout import timeout

In [ ]:
@timeout(5)
def find_list (id, articles, page_num):
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
    params = {'id': id, 'exception_mode' :'recommend', 'page' : page_num}
    BASE_URL = "https://gall.dcinside.com/board/lists"
    resp = requests.get(BASE_URL, params=params, headers=headers)
    soup = BeautifulSoup(resp.content, 'html.parser')
    articles.extend(soup.select(
        'table > tbody > tr > td.gall_tit.ub-word > a'
    ))
    return articles

In [ ]:
from requests.adapters import HTTPAdapter
import lxml.html

GET_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Mobile Safari/537.36",
     }

def gen_session():
    sess = requests.Session()
    sess.mount("https://", HTTPAdapter(max_retries=100))
    sess.headers.update(GET_HEADERS)
    sess.head("https://m.dcinside.com")
    return sess
DEFAULT_SESS = gen_session()

XML_HTTP_REQ_HEADERS = {
    "Accept": "*/*",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Mobile Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9,ko;q=0.8",
    "Cache-Control": "no-cache",
    "Pragma": "no-cache",
    "X-Requested-With": "XMLHttpRequest",
    }
POST_HEADERS = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9,ko;q=0.8",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Content-Type": "application/x-www-form-urlencoded",
    "Pragma": "no-cache",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Mobile Safari/537.36",
    }

TIMEOUT = 3

def comments(board_id, doc_id, sess=DEFAULT_SESS, start_page=1):
    answer = []
    url = "https://m.dcinside.com/ajax/response-comment"
    for page in range(start_page, 999999):
        payload = {"id": board_id, "no": doc_id, "cpage": page, "managerskill":"", "del_scope": "1", "csort": ""}
        res = sess.post(url, headers=XML_HTTP_REQ_HEADERS, data=payload, timeout=TIMEOUT)
        parsed = lxml.html.fromstring(res.text)
        if not len(parsed):
            break
        if not len(parsed[1].xpath("li")): break
        #for li in reversed(parsed[1].xpath("li")):
        for li in parsed[1].xpath("li"):
            if not len(li[0]): continue
#             answer.append({
#                 "id": li.get("no"),
#                 "parent_id": li.get("m_no"),
#                 "author": li[0].text + ("{}".format(li[0][0].text) if li[0][0].text else ""),
#                 "author_id": li[0][1].text if len(li[0]) > 1 else None,
#                 "contents": '\n'.join(i.strip() for i in li[1].itertext()),
#                 "dccon": li[1][0].get("data-original", li[1][0].get("src", None)) if len(li[1]) and li[1][0].tag=="img" else None,
#                 "voice": li[1][0].get("src", None) if len(li[1]) and li[1][0].tag=="iframe" else None,
#                 "time": li[2].text, })
            answer.append('\n'.join(i.strip() for i in li[1].itertext()))
    return answer

In [ ]:
@timeout(3)
def extract_article (id, result, info):
    article = info['href']
    params = {}
    parameters = article.split('?')[1].split('&')
    for param in parameters:
        key, value = param.split('=')
        params[key] = value
    # params = {'id': 'leagueoflegends3', 'exception_mode' :'recommend', 'no': 6396850, 'page' : 1}
    headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
    BASE_URL = "https://gall.dcinside.com/board/view/"
    resp = requests.get(BASE_URL, params=params, headers=headers)
    soup = BeautifulSoup(resp.content, 'html.parser')
    result.add(soup.select('meta:nth-of-type(8)')[0]['content'].split('-')[0])
    result.add(soup.select('meta:nth-of-type(9)')[0]['content'])
    result |= set(comments(id, params['no']))
    return result

In [ ]:
def crawl_dcinside(id: str):
    articles = []
    for i in tqdm(range(100)):
        articles = find_list(id, articles, i)
    
    result = set([])
    for i in range(10):
        for info in tqdm(articles[1000 * i : 1000 * (i+1)]):
            try:
                result = extract_article(id, result, info)
            except:
                continue
            
        with open('./data/data_dc_' + id + '_' + str(i) + '.txt', 'w') as f:
            for sent in result:
                f.write(sent + '\n')

In [ ]:
def crawl_nate():
    url = "https://pann.nate.com"
    talk_url = "https://pann.nate.com/talk"
    resp = requests.get(talk_url)
    html = BeautifulSoup(resp.content, 'html.parser')
    f = open('./data/data_nate.txt', 'w',  encoding='utf-8')
    
    s_talk = html.find('ul', {'id':  'categoryArea', 'class': 'odd'})
    categories = s_talk.findAll('a')
    
    for category in categories:
        f.write("category: {}\n".format(category.getText()))
        href = category['href']
        url_category = url + href
        cat_resp = requests.get(url_category)
        cat_html = BeautifulSoup(cat_resp.content, 'html.parser')

        post_list = cat_html.find('ul', {'class': 'post_list'})
        if(post_list):
            post_lists = post_list.findAll('a')
        
            for p in post_lists:
                href = p['href']
                if(href[6].isalpha()):
                    continue
                resp = requests.get(url+p['href'])
                html = BeautifulSoup(resp.content, 'html.parser')
                post = html.find('div', {'class': 'posting'})
                posts = post.find('div', {'id': 'contentArea'})
                txt = posts.getText().strip()
                if(txt):
                    f.write("post: {}\n".format(txt.strip()))

                f.write('comments: ')
                comments = html.findAll('dd', {'class': 'usertxt'})
                for comment in comments:
                    txt = comment.find('span').getText()
                    if(txt):
                        f.write("{}, ".format(txt.strip()))
    f.close()

In [ ]:
crawl_dcinside('leagueoflegends3')

In [ ]:
crawl_dcinside('iu_new')

In [ ]:
crawl_nate()